In [1]:
import pandas as pd
import json
import psycopg2

In [2]:
FABLABS_API_URL = 'query2_ref.txt'

In [3]:
df = pd.read_csv(FABLABS_API_URL, sep="\n", header=None, names=["Tweets"])

In [4]:
df

,Tweets
0,"{""created_at"":""Fri Apr 04 05:30:11 +0000 2014""..."
1,"{""created_at"":""Fri Apr 04 05:32:57 +0000 2014""..."
2,"{""created_at"":""Fri Apr 04 05:35:06 +0000 2014""..."
3,"{""created_at"":""Fri Apr 04 05:40:22 +0000 2014""..."
4,"{""created_at"":""Fri Apr 04 05:40:26 +0000 2014""..."
...,...
39087,"{""created_at"":""Wed May 28 21:46:19 +0000 2014""..."
39088,"{""created_at"":""Wed May 28 21:49:54 +0000 2014""..."
39089,"{""created_at"":""Wed May 28 21:52:47 +0000 2014""..."
39090,"{""created_at"":""Wed May 28 21:52:54 +0000 2014""..."


In [27]:
def insert_into_table(args, table):
    data = args #json.loads(json.dumps(args[0]))
#     print(data)
    values = list(data.values())
    values = ['' if item is None else item for item in values]
    columns = ','.join(list(data.keys()))
#     print(values)
#     print(columns)
    with psycopg2.connect(dbname='tweet',user='rmohims-admin', host='localhost', password='') as conn:
        with conn.cursor() as cur:
            query_sql = ("insert into {} ({}) VALUES %s;").format(table, columns)
            cur.execute("select id from {} where id = {}".format(table, data['id']))
            duplicate = cur.fetchone()
            conn.commit()
            if duplicate is None:
                cur.execute(query_sql, (tuple(values),))
                conn.commit()
#             print("inserted", type(args[0]))

def transform(args):
    """Placeholder, change, rename, remove... """
    # user tweeted
    user = args["user"]
    
    # retweets and entities from a tweet
    retweeted_status  = args["retweeted_status"] if 'retweeted_status' in args else None
    entities = args["entities"] if 'entities' in args else None
    
    # hastag, urls and user_mentions from tweet
    hash_tag = entities["hashtags"] if 'hashtags' in entities else None
    hash_tag = entities["urls"] if 'urls' in entities else None
    hash_tag = entities["user_mentions"] if 'user_mentions' in entities else None
    
    # a user who retweeted and entity retweeted
    user_retweet = retweeted_status["user"] if retweeted_status is not None and 'user' in retweeted_status else None
    entities_retweet = retweeted_status["entities"] if retweeted_status is not None and 'entities' in retweeted_status else None
    
    # hastag, urls and user_mentions from retweet
    hash_tag_retweet      = entities_retweet["hashtags"] if entities_retweet is not None and 'hashtags' in entities_retweet else None
    urls_retweet          = entities_retweet["urls"] if entities_retweet is not None and 'urls' in entities_retweet else None
    user_mentions_retweet = entities_retweet["user_mentions"] if entities_retweet is not None and 'user_mentions' in entities_retweet else None
    
    del args['user']
    args["users"] = user['id']
    
    if 'display_text_range' in args and args['display_text_range'] is not None:
        del args['display_text_range'] # = ','.join(args['display_text_range'])
    
    # delete coordinates, place, quoted_status, extended_entities, withheld_in_countries, extended_tweet and geo
    args.pop('coordinates', None)
    args.pop('place', None)
    args.pop('quoted_status', None)
    args.pop('extended_entities', None)
    args.pop('withheld_in_countries', None)
    args.pop('extended_tweet', None)
    args.pop('geo', None)

    # tweets
    if retweeted_status is not None:
        args["retweeted_status"] = retweeted_status['id']
        
    if entities is not None:
        entities['entities'] = args['id']
        args.pop('entities')
    
    # entities from tweet
    
    if retweeted_status is not None and 'user' in retweeted_status:
        retweeted_status["user"] = user_retweet['id']
        
        
    if retweeted_status is not None and 'entities' in retweeted_status:
        retweeted_status.pop('entities')
        
    return [user, args] #, retweeted_status, entities, user_retweet, entities_retweet]
    
def load(args):
    """Placeholder, change, rename, remove... """
    tables = ["users", "tweets"] #, "tweets", "entities", "users", "entities"]
    for i, value in enumerate(args):
        insert_into_table(value, tables[i])
    

In [28]:
for col in df.columns:
    print(col)

Tweets


In [29]:
# from sqlalchemy import Table, MetaData, create_engine, text

# engine = create_engine("postgresql://postgres:@localhost/tweet")

# with engine.connect() as conn:
#     conn.execute(text("SET search_path TO test_schema, public"))
#     metadata_obj = MetaData()
#     referring = Table('users', metadata_obj, autoload_with=conn)

In [32]:
i = 0
for x in df.iterrows():
    t = df.iloc[i].to_dict()['Tweets']
    k=json.loads(t)
    l_values = transform(k)
    load(l_values)
    i += 1
#     if i == 10:
#         break